# Utils

> TODO fill in description

In [1]:
#| default_exp utils

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio

In [5]:
#|export
__all__ = ['EventHandler', 'StateHandler', 'StateView', 'Expando', 'await_multiple_events']

In [6]:
#|export
class EventHandler:
    """Subscribable events"""
    def __init__(self):
        self._events = []
    
    def subscribe(self):
        event = asyncio.Event()
        self._events.append(event)
        return event

    def _trigger(self):
        for event in self._events:
            event.set()
        self._events.clear()

In [7]:
#|export
class StateHandler:
    def __init__(self, current_state, state_vals=[True, False]):
        state_vals = list(state_vals) # Can be enums
        self._state_vals = state_vals
        if len(state_vals) != len(set(state_vals)): raise ValueError("`state_vals` must have all unique elements.")
        if current_state not in state_vals: raise ValueError("`current_state` must be in `state_vals`.")
        self.__state_is_on = {state : asyncio.Event() for state in state_vals}
        self.__state_is_on[current_state].set()
        self.__state_is_off = {state : asyncio.Event() for state in state_vals}
        self._current_state = current_state
        for state in self.__state_is_off:
            if state != current_state: self.__state_is_off[state].set()
        
    def check(self, state):
        return self.__state_is_on[state].is_set()
    
    def get(self):
        return self._current_state
    
    def set(self, state):
        if state not in self._state_vals: raise ValueError(f"Invalid state: {state}. Possible states: {', '.join(self._state_vals)}")
        self._current_state = state
        for _state in self.__state_is_on:
            if _state == state:
                self.__state_is_on[_state].set()
                self.__state_is_off[_state].clear()
            else:
                self.__state_is_on[_state].clear()
                self.__state_is_off[_state].set()
            
    def wait(self, state, state_value=True):
        if state_value: return self.__state_is_on[state].wait()
        else: return self.__state_is_off[state].wait()
        
    async def __event_func(self, state, event):
        await state.wait()
        event.set()
      
    def get_state_event(self, state, state_value=True):
        if state_value: return self.__state_is_on[state]
        else: return self.__state_is_off[state]
        
    def get_state_toggle_event(self, state, state_value=True):
        event = asyncio.Event()
        if state_value: asyncio.run(self.__event_func(self.__state_is_on[state], event))
        else: asyncio.run(self.__event_func(self.__state_is_off[state], event))
        return event
    
    def __str__(self):
        return f"State: {self._current_state}"

In [ ]:
#|export
class StateView:
    def __init__(self, state_handler):
        self._state_handler: StateHandler = state_handler
        
    def check(self, state):
        return self._state_handler.check(state)
    
    def get(self):
        return self._state_handler._current_state
            
    def wait(self, state, state_value=True):
        return self._state_handler.wait(state, state_value)
      
    def get_state_event(self, state, state_value=True):
        return self._state_handler.get_state_event(state, state_value)
        
    def get_state_toggle_event(self, state, state_value=True):
        return self._state_handler.get_state_toggle_event(state, state_value)
    
    def __str__(self):
        return f"State: {self._state_handler._current_state}"

In [10]:
#|export
class Expando:
    def __init__(self, _attrs=None, obj_name="Expando"):
        self.idx = ()
        self._attrs = dict(_attrs) if _attrs is not None else {}
        self._obj_name = obj_name
        
    def __getattr__(self, key):
        return self[key]
        
    def __getitem__(self, key):
        if key in self._attrs.keys():
            return self._attrs[key]
        else:
            #TODO proper exception in Expando.__getitem__
            raise Exception(f"{key} does not exist")
        
    def keys(self):
        return self._attrs.keys()
    
    def values(self):
        return self._attrs.values()
        
    def __iter__(self):
        return self._attrs.__iter__()
    
    def __contains__(self, key):
        return key in self._attrs

    def __len__(self):
        return self._attrs.__len__()
    
    def __str__(self):
        return f'{self.obj_name}: {", ".join([f"{k}: {v}" for k,v in self._ports.items()])}'

In [11]:
#|export
async def await_multiple_events(*events):
    while not all([event.is_set() for event in events]): # In the off-chance that as asyncio.wait finishes, one of the events is cleared
        event_await_tasks = []
        for event in events:
            async def await_event(): await event.wait()
            event_await_tasks.append(asyncio.create_task(await_event))
        await asyncio.wait(event_await_tasks)